This notebook contains an algorithm for best subset selection for linear regression using several selction criteria.

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
from itertools import combinations
import pandas as pd

Here are the formulas for various selection criteria for the best subset in a linear regression model.

Total sum of squares, `SSTO`:
$$
SSTO = \sum (Y_i - \bar{Y})^2
$$

Error sum of squares, `SSE`:
$$
SSE = \sum (Y_i - \hat{Y_i})^2
$$

Regression sum of squares, `SSR` (not used in calculations, but included for reference):  
$$
SSR = \sum (\hat{Y_i} - \bar{Y})^2
$$

Relationship between `SSTO`, `SSE`, and `SSR`:
$$
SSTO = SSE + SSR
$$

Mean squared error, `MSE`:
$$
MSE = \frac{SSE}{n-2}
$$

-----

-----

The `p` in the following formulas referes to the subset of `p` variables from the original set of independent variables. For example, if the original `X` has variables `x1`, `x2`, and `x3`, for `p=2`, `Xp` would be `{x1, x2}`, `{x1, x3}`, and `{x2, x3}`, and the criterions would be based on those subsets.

Coefficient of multiple determination, `R2`:
$$
R^{2}_p = 1 - \frac{SSE_p}{SSTO}
$$

Adjusted coefficient of multiple determination, `adj_R2`:  

$$
R^2_{a,p} = 1 - \left(\frac{n-1}{n-p}\right)\frac{SSE_p}{SSTO} = 1 - \frac{MSE_p}{\frac{SSTO}{n-1}}
$$

Mallows's `Cp`:
$$
C_p = \frac{SSE_p}{MSE(X_{1},...,X_{p-1})} - (n-2p)
$$

The following functions will calculate different statistical values.

In [3]:
def SSTO(y):
    
    '''Calculates sum of squares from the mean.'''
    
    y_mean = np.mean(y)
    squared_errors = (y - y_mean)**2
    
    return np.sum(squared_errors)

In [4]:
def SSE(y, predictions):
    
    '''Calculates sum of squared errors between predictions and actual values.'''
    
    squared_errors = (y - predictions)**2
    
    return np.sum(squared_errors)

In [5]:
def adj_R2(_sse, _ssto, n, p):
    
    '''Calculates the adjusted R^2.'''
    
    return 1 - (n-1)/(n-p) * _sse/_ssto

In [6]:
def Cp(sse_p, sse_P, n, p, P):
    
    '''Calculates Mallows's Cp value. Needs sse_p and sse_P to be pre-calculated.'''
    
    return sse_p / (sse_P/(n-P)) - (n - 2*p)

In [7]:
def AIC(_sse, n, p):
    
    '''Calculates the Akaike information criterion'''
    
    return n * np.log(_sse) - n * np.log(n) + 2*p

In [8]:
def SBC(_sse, n, p):
    
    '''Calculates Schwarz Bayesian criterion'''
    
    return n * np.log(_sse) - n * np.log(n) + np.log(n) * p

In [9]:
def PRESS(X, y):
    
    '''Calculates PRESS criterion.'''
    
    lr = LinearRegression()
    pred = np.zeros(y.shape)
    
    for i in range(X.shape[0]):
        y_mod = np.delete(y, i, 0)
        X_mod = np.delete(X, i, 0)
        lr.fit(X_mod, y_mod)
        pred[i] = lr.predict(X[i].reshape(1, -1))
        
    return SSE(y, pred)

Define some objects that will be needed in the main function.

In [10]:
# DataFrames that will store the best subset related information
best_values_df = pd.DataFrame(columns = ['p', 'SSEp', 'R^2_p', 'Adj. R^2_p',
                                         'Cp', 'AICp', 'SBCp', 'PRESSp'])
best_subsets_df = pd.DataFrame(columns = ['p', 'SSEp', 'R^2_p', 'Adj. R^2_p',
                                         'Cp', 'AICp', 'SBCp', 'PRESSp'])

In [11]:
# the main function that will use the criterion calculations
# to determine the best subsets for regression
def get_subsets(X, y, P):
    # make sure that both X and y are numpy arrays
    if (type(X) != np.ndarray) or (type(y) != np.ndarray):
        raise TypeError('X and y must be numpy arrays')
    
    # check to makes sure we have the same number of rows in X and y
    if X.shape[0] != y.shape[0]:
        raise ValueError('X and y must have the same number of rows')
        
    # set n as the number of observations
    n = X.shape[0]
    
    # create a range of values 1 through P for the numbers of variables in the subsets
    P_range = range(1, P+1)
    
    # for both dataframes best_values_df and best_subsets_df,
    # set values in the 'p' column to P_range values, and set that column as the index
    best_values_df['p'] = P_range
    best_values_df.set_index('p', inplace = True)
    best_subsets_df['p'] = P_range
    best_subsets_df.set_index('p', inplace = True)
    
    # create subsets of X consisting of 1 through P variables
    # first, create an empty list to hold the tuples of subsets
    X_subsets = []
    
    # create combinations of subsets using the 'combinations' function
    # and iterating over values in range equal to the number of X variables
    for i in range(1, P):
        combs = combinations(range(X.shape[1]), i)
        #TODO: print and verify subsets
        for item in combs:
            X_subsets.append(item)
            
    # create intermediate dataframes to hold criterion values
    SSE_df = pd.DataFrame(columns=['X_var', 'p', 'SSEp'])
    SSE_df.set_index('X_var')
    R2_df = pd.DataFrame(columns=['X_var', 'p', 'R2p'])
    adj_R2_df = pd.DataFrame(columns=['X_var', 'p', 'adj_R2p'])
    C_df = pd.DataFrame(columns=['X_var', 'p', 'Cp', 'Abs_Cp'])
    AIC_df = pd.DataFrame(columns=['X_var', 'p', 'AICp'])
    SBC_df = pd.DataFrame(columns=['X_var', 'p', 'SBCp'])
    PRESS_df = pd.DataFrame(columns=['X_var', 'p', 'PRESSp'])
    
    # calculate SSTO for y
    # SSTO will be the same for any subset of X (look at the formula above for details)
    _ssto = SSTO(y)
    
    # Scikit-learn linear regression used in calculations
    lin_reg = LinearRegression()
    
    # populate SSE, R2, adj_R2, AIC, and SBC values in the respective dataframes
    for i in P_range:
        # p = 1 means using just the constant with no subset of X.
        # Hence, all entries for X_var are 'None'.
        if i == 1:
            _sse = _ssto
            SSE_df.loc['None'] = ['None', i, _sse]
            R2_df.loc['None'] = ['None', i, 0]
            adj_R2_df.loc['None'] = ['None', i, 0]
            AIC_df.loc['None'] = ['None', i, AIC(_sse, n, i)]
            SBC_df.loc['None'] = ['None', i, SBC(_sse, n, i)]
        else:
            # get only subsets that consist only of i-1 variables
            current_subset = [item for item in X_subsets if len(item) == i-1]
            # calculate criterions for the current_subset
            for item in current_subset:
                # fit linear regression to the current subset
                # and use it for predict values when needed
                lin_reg.fit(X[:, item], y)
                y_hat = lin_reg.predict(X[:, item])
                # create var numbers (X1, X2, etc) for dataframe display
                var_numbers = [f'X{num + 1}' for num in item]
                # populate SSE_df
                _sse = SSE(y, y_hat)
                # convert the list of var_numbers into a single string to use for indexing
                # e.g. a list of ['X1', 'X2'] turns into a string "['X1', 'X2']"
                # which can be used a single label
                SSE_df.loc[str(var_numbers)] = [var_numbers, i, _sse]
                # populate R2_df using scikit-learn's 'score' method
                R2_df.loc[str(var_numbers)] = [var_numbers, i, lin_reg.score(X[:, item], y)]
                R2_df.R2p = R2_df.R2p.astype('float')
                # populate the adj_R2, AIC, and SBC dataframes
                adj_R2_df.loc[str(var_numbers)] = [var_numbers, i, adj_R2(_sse, _ssto, n, i)]
                adj_R2_df.adj_R2p = adj_R2_df.adj_R2p.astype('float')
                AIC_df.loc[str(var_numbers)] = [var_numbers, i, AIC(_sse, n, i)]
                SBC_df.loc[str(var_numbers)] = [var_numbers, i, SBC(_sse, n, i)]
                
    # Calculate Cp values and populate C_df
    for i in P_range:
        if i == 1:
            # get SSEp value for the the whole set of variables
            # which can be extracted from SSE_df using the last var_numbers values
            sse_P = SSE_df.loc[str(var_numbers)].SSEp
                
            # calculate Cp values using sse_P and _ssto
            # (_ssto is used in place of sse_p since 
            # we're not using a subset of X for this specific calculation)
            Cp_val = Cp(_ssto, sse_P, n, i, P)
                
            # enter the value into C_df
            #TODO: why is it 'abs(Cp_val - i)'?
            C_df.loc['None'] = ['None', i, Cp_val, abs(Cp_val - i)]
        else:
            current_subset = [item for item in X_subsets if len(item) == i - 1]
            for x in current_subset:
                # create variable names such as 'X1', 'X2', etc
                c_var_numbers = [f'X{num+1}' for num in x]
                    
                # get the _sse value for the current var number
                _sse = SSE_df.loc[str(c_var_numbers)].SSEp
                    
                # calculate Cp value
                Cp_val = Cp(_sse, sse_P, n, i, P)
                    
                # populate C_df; #TODO: why 'abs(Cp_val - i)'?
                C_df.loc[str(c_var_numbers)] = [c_var_numbers, i, Cp_val, abs(Cp_val - i)]
                    
    # calculate PRESSp and populate PRESS_df
    PRESS_predictions = np.zeros(y.shape)
    for i in P_range:
        if i == 1:
            for j in range(X.shape[0]):
                # delete a set of y values to be replaced by predictions
                y_mod = np.delete(y, j, 0)
                # in case of no X variables (P=1), use the mean as the prediction
                PRESS_predictions[j] = np.mean(y_mod)
            PRESS_df.loc['None'] = ['None', i, SSE(y, PRESS_predictions)]
        else:
            current_subset = [item for item in X_subsets if len(item) == i-1]
            for x in current_subset:
                PRESS_var_numbers = [f'X{num+1}' for num in x]
                PRESS_df.loc[str(PRESS_var_numbers)] = [PRESS_var_numbers, i,
                                                        PRESS(X[:,x], y)]
                    
    for i in P_range:
        best_values_df.loc[i, 'SSEp'] = SSE_df[SSE_df.p == i].min().SSEp
        best_subsets_df.loc[i, 'SSEp'] = SSE_df[SSE_df.p == i].SSEp.idxmin()
        best_values_df.loc[i, 'R^2_p'] = R2_df[R2_df.p == i].max().R2p
        best_subsets_df.loc[i, 'R^2_p'] = R2_df[R2_df.p == i].R2p.idxmax()
        best_values_df.loc[i, 'Adj. R^2_p'] = adj_R2_df[adj_R2_df.p == i].max().adj_R2p
        best_subsets_df.loc[i, 'Adj. R^2_p'] = adj_R2_df[adj_R2_df.p == i].adj_R2p.idxmax()
        best_values_df.loc[i, 'Cp'] = C_df[C_df.p == i].min().Cp
        best_subsets_df.loc[i, 'Cp'] = C_df[C_df.p == i].Abs_Cp.idxmin()
        best_values_df.loc[i, 'AICp'] = AIC_df[AIC_df.p == i].min().AICp
        best_subsets_df.loc[i, 'AICp'] = AIC_df[AIC_df.p == i].AICp.idxmin()
        best_values_df.loc[i, 'SBCp'] = SBC_df[SBC_df.p == i].min().SBCp
        best_subsets_df.loc[i, 'SBCp'] = SBC_df[SBC_df.p == i].SBCp.idxmin()
        best_values_df.loc[i, 'PRESSp'] = PRESS_df[PRESS_df.p == i].min().PRESSp
        best_subsets_df.loc[i, 'PRESSp'] = PRESS_df[PRESS_df.p == i].PRESSp.idxmin()
        
    display('Best Values for Criteria')
    display(best_values_df)
    print('\n')
    display('Best Subsets for Criteria')
    display(best_subsets_df)

In [12]:
np.set_printoptions(suppress=True)
data = np.loadtxt('Surgical Data.txt')
print(data)

[[   6.7     62.      81.       2.59    50.       0.       1.       0.
   695.       6.544]
 [   5.1     59.      66.       1.7     39.       0.       0.       0.
   403.       5.999]
 [   7.4     57.      83.       2.16    55.       0.       0.       0.
   710.       6.565]
 [   6.5     73.      41.       2.01    48.       0.       0.       0.
   349.       5.854]
 [   7.8     65.     115.       4.3     45.       0.       0.       1.
  2343.       7.759]
 [   5.8     38.      72.       1.42    65.       1.       1.       0.
   348.       5.852]
 [   5.7     46.      63.       1.91    49.       1.       0.       1.
   518.       6.25 ]
 [   3.7     68.      81.       2.57    69.       1.       1.       0.
   749.       6.619]
 [   6.      67.      93.       2.5     58.       0.       1.       0.
  1056.       6.962]
 [   3.7     76.      94.       2.4     48.       0.       1.       0.
   968.       6.875]
 [   6.3     84.      83.       4.13    37.       0.       1.       0.
   745.  

In [13]:
data_x = data[:,:-1]
data_y = data[:, -1]
print(data_x)
print(data_y)

[[   6.7    62.     81.      2.59   50.      0.      1.      0.    695.  ]
 [   5.1    59.     66.      1.7    39.      0.      0.      0.    403.  ]
 [   7.4    57.     83.      2.16   55.      0.      0.      0.    710.  ]
 [   6.5    73.     41.      2.01   48.      0.      0.      0.    349.  ]
 [   7.8    65.    115.      4.3    45.      0.      0.      1.   2343.  ]
 [   5.8    38.     72.      1.42   65.      1.      1.      0.    348.  ]
 [   5.7    46.     63.      1.91   49.      1.      0.      1.    518.  ]
 [   3.7    68.     81.      2.57   69.      1.      1.      0.    749.  ]
 [   6.     67.     93.      2.5    58.      0.      1.      0.   1056.  ]
 [   3.7    76.     94.      2.4    48.      0.      1.      0.    968.  ]
 [   6.3    84.     83.      4.13   37.      0.      1.      0.    745.  ]
 [   6.7    51.     43.      1.86   57.      0.      1.      0.    257.  ]
 [   5.8    96.    114.      3.95   63.      1.      0.      0.   1573.  ]
 [   5.8    83.     88.  

In [15]:
data_y.reshape(54,1)

array([[6.544],
       [5.999],
       [6.565],
       [5.854],
       [7.759],
       [5.852],
       [6.25 ],
       [6.619],
       [6.962],
       [6.875],
       [6.613],
       [5.549],
       [7.361],
       [6.754],
       [6.554],
       [6.695],
       [6.526],
       [5.321],
       [6.309],
       [6.731],
       [5.883],
       [5.866],
       [6.395],
       [6.332],
       [6.478],
       [6.621],
       [6.302],
       [7.583],
       [6.167],
       [6.396],
       [6.094],
       [5.198],
       [6.019],
       [6.944],
       [6.179],
       [6.453],
       [6.519],
       [5.893],
       [6.457],
       [6.558],
       [6.283],
       [6.366],
       [7.147],
       [6.288],
       [6.178],
       [6.416],
       [6.867],
       [7.17 ],
       [6.365],
       [6.983],
       [6.005],
       [6.361],
       [6.31 ],
       [6.478]])

In [16]:
get_subsets(data_x, data_y, 9)

'Best Values for Criteria'

,SSEp,R^2_p,Adj. R^2_p,Cp,AICp,SBCp,PRESSp
p,,,,,,,
1,12.8077,0,0,496.346,-75.7025,-73.7135,13.2956
2,1.78036,0.860993,0.85832,26.2239,-180.257,-176.279,2.34979
3,1.51539,0.881682,0.877042,16.8795,-186.959,-180.992,2.1786
4,1.22789,0.904129,0.898377,6.57044,-196.319,-188.363,1.78989
5,1.15349,0.909938,0.902586,5.3852,-197.694,-187.749,1.70219
6,1.10277,0.913898,0.904929,5.21386,-198.122,-186.188,1.69184
7,1.07542,0.916034,0.905315,6.04252,-197.479,-183.556,1.71034
8,1.05195,0.917866,0.905368,7.03771,-196.67,-180.759,1.7231
9,1.03423,0.91925,0.904894,8.2791,-195.588,-177.687,1.79334


'Best Subsets for Criteria'

,SSEp,R^2_p,Adj. R^2_p,Cp,AICp,SBCp,PRESSp
p,,,,,,,
1,None,None,None,None,None,None,None
2,['X9'],['X9'],['X9'],['X9'],['X9'],['X9'],['X9']
3,"['X3', 'X9']","['X3', 'X9']","['X3', 'X9']","['X3', 'X9']","['X3', 'X9']","['X3', 'X9']","['X3', 'X9']"
4,"['X2', 'X3', 'X9']","['X2', 'X3', 'X9']","['X2', 'X3', 'X9']","['X2', 'X3', 'X9']","['X2', 'X3', 'X9']","['X2', 'X3', 'X9']","['X2', 'X3', 'X9']"
5,"['X2', 'X3', 'X8', 'X9']","['X2', 'X3', 'X8', 'X9']","['X2', 'X3', 'X8', 'X9']","['X2', 'X3', 'X8', 'X9']","['X2', 'X3', 'X8', 'X9']","['X2', 'X3', 'X8', 'X9']","['X2', 'X3', 'X8', 'X9']"
6,"['X2', 'X3', 'X6', 'X8', 'X9']","['X2', 'X3', 'X6', 'X8', 'X9']","['X2', 'X3', 'X6', 'X8', 'X9']","['X2', 'X3', 'X5', 'X8', 'X9']","['X2', 'X3', 'X6', 'X8', 'X9']","['X2', 'X3', 'X6', 'X8', 'X9']","['X2', 'X3', 'X6', 'X8', 'X9']"
7,"['X2', 'X3', 'X5', 'X6', 'X8', 'X9']","['X2', 'X3', 'X5', 'X6', 'X8', 'X9']","['X2', 'X3', 'X5', 'X6', 'X8', 'X9']","['X2', 'X3', 'X4', 'X6', 'X8', 'X9']","['X2', 'X3', 'X5', 'X6', 'X8', 'X9']","['X2', 'X3', 'X5', 'X6', 'X8', 'X9']","['X2', 'X3', 'X5', 'X6', 'X8', 'X9']"
8,"['X2', 'X3', 'X5', 'X6', 'X7', 'X8', 'X9']","['X2', 'X3', 'X5', 'X6', 'X7', 'X8', 'X9']","['X2', 'X3', 'X5', 'X6', 'X7', 'X8', 'X9']","['X2', 'X3', 'X4', 'X5', 'X6', 'X8', 'X9']","['X2', 'X3', 'X5', 'X6', 'X7', 'X8', 'X9']","['X2', 'X3', 'X5', 'X6', 'X7', 'X8', 'X9']","['X2', 'X3', 'X5', 'X6', 'X7', 'X8', 'X9']"
9,"['X1', 'X2', 'X3', 'X5', 'X6', 'X7', 'X8', 'X9']","['X1', 'X2', 'X3', 'X5', 'X6', 'X7', 'X8', 'X9']","['X1', 'X2', 'X3', 'X5', 'X6', 'X7', 'X8', 'X9']","['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9']","['X1', 'X2', 'X3', 'X5', 'X6', 'X7', 'X8', 'X9']","['X1', 'X2', 'X3', 'X5', 'X6', 'X7', 'X8', 'X9']","['X1', 'X2', 'X3', 'X5', 'X6', 'X7', 'X8', 'X9']"
